<a href="https://colab.research.google.com/github/BZoennchen/musical-interrogation/blob/main/partII/melody_mc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Die folgenden 6 Zellen sind für die Ausführung im Colab nötig.

In [ ]:
#@title clone git repository
%%capture
!rm -rf musical-interrogation
!git clone https://github.com/BZoennchen/musical-interrogation.git

In [ ]:
#@title move into directory
%%capture
import os
import zipfile
os.chdir('musical-interrogation/partII')

In [ ]:
#@title install dependencies to play sound
%%capture
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
print('done!')

In [ ]:
#@title install dependencies to show score in music notation
%%capture
print('installing musescore3...')
!apt-get install musescore3 > /dev/null
print('done!')

In [ ]:
#@title install python libs
%%capture
!pip install torch music21 matplotlib fluidsynth midi2audio

In [ ]:
#@title download and unzip dataset
%%capture
!wget https://syncandshare.lrz.de/dl/fiRdUknKuVaq3sjT2PNZax/erk.zip
with zipfile.ZipFile('./erk.zip', 'r') as zip_ref:
    zip_ref.extractall('./../deutschl/')

# Markov-Kette (erster Ordnung)

**AICA Crashkurs, Dr. Benedikt Zönnchen**

Eine Möglichkeit sich der Problemstellung der Melodiegenerierung zu widmen ist es die bedingte Wahrscheinlichkeit einer bestimmten Note $e_t$ zu bestimmen, welche nach einer bestimmte Sequenz von Noten $e_{1}, \ldots, e_{t-1}$ folgt:

$$P(X_t = e_t | X_{t-1} = e_{t-1},\ldots,X_1 = e_{1}).$$

Kennen wir die Wahrscheinlichkeiten für jede erdenkliche Note, so können wir von dieser Wahrscheinlichkeitsverteilung ziehen!

In diesem Notebook betrachten wir lediglich eine Sequenzlänge von 1, d.h.,

$$P(X_t = e_t | X_{t-1} = e_{t-1}).$$

Wir schätzen diese Wahrscheinlichkeit über die Frequenz der Übergänge von einer Note zur anderen, d.h., wir zählen wie oft ein bestimmter Übergang vorkommt und teilen durch die gesamte Anzahl der Übergänge für eine bestimmte Note.

Wir erhalten eine sog. *Markov Matrix* $\mathbf{P}$, deren Zeile $i$ die Wahrscheinlichkeitsverteilung für eine die Note $i$ angibt. Eintrag $j$ in Zeile $i$ gibt demnach die Wahrscheinlichkeit für den Übergang der Note $i$ nach $j$ an.

In [ ]:
import sys
sys.path.append("..") # not required in colab

import matplotlib.pyplot as plt
import music21 as m21
import torch
from preprocess import load_songs_in_kern, NoteEncoder, StringToIntEncoder, TERM_SYMBOL

from utils import score_to_wav
from IPython.display import Audio

torch.manual_seed(0);

## 1. Datenvorbereitung

Zunächst laden wir unsere (1700 **einstimmigen**) Musikstücke aus denen wir die Frequenzen berechnen wollen.

In [ ]:
# this takes a while
scores = load_songs_in_kern('./../deutschl/erk')

Wir können uns eines der Musikstücke anhören oder auch anzeigen lassen.

In [ ]:
Audio(score_to_wav(scores[0], 'score1.wav'))

In [ ]:
scores[0].show()

Als nächstes verwandeln wir die Noten in gut leserliche Zeichenketten, wobei jede Note durch genau eine Zeichenkette repräsentiert wird und zwar der Form ``MIDI-Tonhöhe/Länge in vielfaches von time_step``.

Dies übernimmt der ``NoteEncoder``. Dieser transponiert die Musikstücke zusätzlich nach C-Dur.
Zusätzlich filtert er Musikstücke heraus, welche wir mit unserem ``time_step`` nicht abbilden können.
Z.B. wenn ``time_step = 1/8`` dann können wir keine ``1/16``-Noten oder auch ``1/8 + 1/16``-Noten abbilden. 

In [ ]:
# this takes a while
time_step = 1/8
print(f'one timestep represents {time_step} beats')

encoder = NoteEncoder(time_step)
enc_songs, invalid_song_indices = encoder.encode_songs(scores)

print(f'there are {len(enc_songs)} valid songs and {len(invalid_song_indices)} songs')

In [ ]:
' '.join(enc_songs[0])

In [ ]:
print(f'longest melody: {max(len(m) for m in enc_songs)}')
print(f'shortest melody: {min(len(m) for m in enc_songs)}')

Da der Computer besser mit Zahlen umgehen kann bauen wir uns eine Abbildung von den jeweiligen Zeichenketten zu Zahlen $$\{0, 1, 2, \ldots, m-1\}$$ und umgekehrt. Dies übernimmt ``StringToIntEncoder``:

In [ ]:
string_to_int = StringToIntEncoder(enc_songs)
print(f'number of unique symbols: {len(string_to_int)}')
encoded_symbol = string_to_int.encode(enc_songs[0][0])
print(f'symbol {enc_songs[0][0]} is encoded to {encoded_symbol}')
print(f'encoded symbol {encoded_symbol} is decoded to {string_to_int.decode(encoded_symbol)}')

## 2. Konstruktion der Markov-Matrix

Sei $m$ die Anzahl der verschiedenen Noten, so benötigen wir eine Matrix $\mathbf{P}$ mit $m$ Zeilen und $m$ Spalten.
Wir erstellen erst eine Matrix $\mathbf{N}$ um die Übergänge zu zählen. Wir benötigen noch zwei spezielle Übergänge:

1. Keine Note zu Note (für den Anfang eines Musikstücks)
2. Note zu keine Note (für den Ende)

Dafür verwenden wir ein spezielles Symbol ``TERM_SYMBOL``, welches der ``StringToIntEncoder`` bereits berücksichtigt.

In [ ]:
# create an matrix m times m matrix containing zeros.
m = len(string_to_int)
N = torch.zeros((m, m)) 

# for each note/event e_i followed by e_j increase the matrix component n_ij by 1
for enc_song in enc_songs:
    chs = [TERM_SYMBOL] + enc_song + [TERM_SYMBOL]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = string_to_int.encode(ch1)
        ix2 = string_to_int.encode(ch2)
        N[ix1, ix2] += 1

Im zweiten Schritt berechnen wir die Markov-Matrix $P$ indem wir die Häufigkeit der Übergänge durch die Zahl der Gesamtenübergänge dividieren. D.h. wir dividieren jeden Eintrag der Matrix durch die Summe der jeweiligen Zeile.

In [ ]:
P = N.float()
P = P / P.sum(dim=1, keepdim=True)

Wir können diese große Matrix auch visualisieren und sehen, dass sehr viele Einträge 0 sind.

In [ ]:
plt.rcParams.update({'font.size': 6})
plt.figure(figsize=(8, 8))
plt.imshow(P, cmap='Blues')

Wir sehe in der Visualisierung, dass die Wahrscheinlichkeiten in der Nähe der Diagonalen groß sind. Dies liegt daran, dass die Matrix nach Tonhöhen sortiert ist und große Sprünge in Melodien unüblich sind. Zusätzlich sehen wir eine Spalte (weit rechts) und eine Zeile (weit unten) mit höheren Wahrscheinlichkeiten. Dies ist vermutlich die Spalte die **Note nach keine Note** bzw. **Keine Note nach Note** repräsentiert.

## 3. Melodiegenerierung

Gegeben einer Sequenz beliebiger Länge, dient die Funktion ``generate`` der Generierung eines neues neuen Musikstücks.

Mit $\mathbf{P}$ können wir nun neue Melodien generieren.
Wir starten mit dem speziellen Symbol $e_1$ = ``TERM_SYMBOL`` ("Beginn des Stücks") als unsere erste Note/Event $e_1$. Dann berechnen wir $e_2$ aus der bedingten Verteilung $P(X_2 = e_2 | X_1 = e_1)$, gegeben durch die Zeile, die zu $e_1$ gehört. Als nächstes berechnen wir $e_3$ durch $P(X_2 = e_3 | X_1 = e_2)$.
Diesen Vorgang führen wir so lange fort bis wir irgendwann ``TERM_SYMBOL`` ziehen.

``temperature`` bestimmt wie stark die vom Modell gelernte Wahrscheinlichkeitsverteilung beachtet wird.

+ ``temperature`` gleich 1.0 bedeutet, dass von der Wahrscheinlichkeitsverteilung gesampelt wird.
+ ``temperature`` gegen unendlich bedeutet, dass gleichverteilt gesampelt wird (mehr Variation)
+ ``temperature`` gegen 0 bedeutet, dass die hohe Wahrscheinlichkeiten verstärkt werden (weniger Variation)

Sie können eine maximale Länge des Stücks festlegen und auch einen Anfang eines Stücks mitliefern.

In [ ]:
def next_event_number(char:str, temperature:float):
    distribution = P[string_to_int.encode(char)] / temperature
    ix = torch.multinomial(distribution, num_samples=1, replacement=True).item()
    return ix

In [ ]:
def generate(seq: list[str]=None, max_len:int=None, temperature:float=1.0):
    generated_encoded_song = []
    if seq != None:
        generated_encoded_song = seq.copy()
    char = TERM_SYMBOL
    while max_len == None or max_len > len(generated_encoded_song):
        ix = next_event_number(char, temperature)
        char = string_to_int.decode(ix)
        if char == TERM_SYMBOL:
            break
        generated_encoded_song.append(char)
    return generated_encoded_song

In [ ]:
# number of songs we want to generate
n_scores = 5

generated_encoded_songs = []
for _ in range(n_scores):
    encoded_song = generate(max_len=120)
    print(f'generated {" ".join(encoded_song)} conisting of {len(encoded_song)} notes')
    generated_encoded_songs.append(encoded_song)

## 4. Datennachbearbeitung

Wir wollen uns die generierten Stücke natürlich anhören. Der ``NoteEncoder`` kann dies für uns übernehmen.

In [ ]:
generated_scores = encoder.decode_songs(generated_encoded_songs)

In [ ]:
Audio(score_to_wav(generated_scores[0], 'g_score1.wav'))

In [ ]:
generated_scores[0].show()

# Fragen

+ Es scheint so als ließe sich diese Methode verbessern indem wir Markov-Ketten höherer Ordnung verwenden. Welche Probleme treten dann auf?